In [4]:
import pandas as pd
import numpy as np
import re
import datetime
pd.options.display.max_columns = None

Seguiremos trabajando con mismos csv que en el pair anterior, el de los empleados que unimos en el pair de unión de datasets y el de los tickets de todos los años. Para ello, lo primero que tendremos que hacer es cargar estos dos csv.

In [5]:
df = pd.read_csv('../datos/historico_tickets.csv', index_col = 0)
df.sample(3)

,id_ticket,fecha,id_empleado,id_agente,categoría,tipo,severidad,prioridad,días_resolución,satisfacción,semestre,año
12628,TWLEER-1242673586,2016-10-30,428,5,Software,Problema,2 - Normal,1 - Baja,2,5,2º semestre,2016
11918,SDLTSR-2843432770,2018-11-28,337,7,Sistema,Solicitud,2 - Normal,0 - Sin Asignar,0,4,2º semestre,2018
19487,SHLTET-0344076332,2020-09-02,373,23,Hardware,Solicitud,2 - Normal,3 - Alta,8,5,2º semestre,2020


1- Cread una columna nueva y una función que nos de el mes en el que se solicitó el servicio. Tened en cuenta que todas las filas tienen la misma estructura. A la hora de escribir la función debemos tener en cuenta que puede que no haya la información de mes en la columna de la fecha. En ese caso devolved un nulo (NaN).

📌 Nota La fecha tendrá que ir formato de string como "Enero", "Febrero", etc.

💡 Pista 💡Podréis usar regex, pero no es estrictamente necesario.

In [6]:
df['fecha'] = df['fecha'].apply(pd.to_datetime)

In [7]:
def sacar_mes(f):
    return f.strftime("%B")

In [8]:
df['mes'] = df["fecha"].apply(sacar_mes)

In [9]:
df.sample(3)

,id_ticket,fecha,id_empleado,id_agente,categoría,tipo,severidad,prioridad,días_resolución,satisfacción,semestre,año,mes
8700,KHLTSR-3843245223,2018-05-25,582,42,Sistema,Solicitud,2 - Normal,3 - Alta,8,4,1 semestre,2018,May
6749,KMLTER-7542606336,2016-08-24,1063,23,Software,Solicitud,2 - Normal,2 - Media,5,5,2º semestre,2016,August
660,GDLTER-9643247867,2018-05-27,496,48,Software,Solicitud,4 - Crítico,0 - Sin Asignar,6,5,1 semestre,2018,May


2- En la tabla de empleados, tenemos los nombres y los apellidos de los empleados, pero en columnas separadas. En los ejercicios de pair de la lección del groupby vimos que si quisieramos agrupar por los nombres, esto se nos puede complicar. Ahora nos piden que creemos una columna donde tengamos el nombre y el apellido juntos.

📌 NOTA La columna nueva debe tener el nombre y apellido con la primera letra en mayúsculas, por ejemplo "Jesús Villa".

📌 NOTA Una vez hallamos creado la columna nueva, eliminad las columnas de nombre y apellido originales.

In [10]:
dfe = pd.read_csv('../datos/empleados_puestos.csv', index_col = 0)
dfe.sample(3)

,Turno,ID Empleado,Apellido,Nombre,ID Puesto,Email,Puesto,Departamento,Tipo Puesto
585,NaN,158,Moreno,Lizeth,55-23200,lizeth.moreno@tractchun.com,Ingeniero de Manufactura,INGENIERIA DE MANUFACTURA,Operativo
931,NaN,52,CAMPAS,ANA,20-31320,ana.campas@tractchun.com,Clerk de Aduanas,"SISTEMAS, TRAFICO Y ADUANAS",Administrativo
1659,NaN,818,Restrepo,Olga,30-16000,olga.restrepo @tractchun.com,Supervisor General de Areas,"AUDITORIA INTERNA, CALIBRACION Y METROLOGIA",Mandos Medios


In [11]:
def nombre_completo(nombre, apellido):
    try:
        nc = nombre.title() + ' ' + apellido.title()
        return nc
    except:
        return np.nan

In [12]:
dfe['Nom_Completo'] = dfe.apply(lambda dataf: nombre_completo(dataf['Nombre'], dataf['Apellido']), axis = 1)

In [13]:
dfe.drop('Apellido', axis = 1, inplace = True)
dfe.drop('Nombre', axis = 1, inplace = True)

In [14]:
dfe.sample(3)

,Turno,ID Empleado,ID Puesto,Email,Puesto,Departamento,Tipo Puesto,Nom_Completo
251,NaN,1547,15-24100,leonor.torrico@tractchun.com,Supervisor de Producción,PRODUCCION,Mandos Medios,Leonor Torrico
1539,NaN,1998,90-14100,francisca.bribiesca@tractchun.com,Ingeniero de Calidad,INGENIERIA DE CALIDAD,Operativo,Francisca Bribiesca
287,NaN,638,15-24100,grijalva.ochoa@tractchun.com,Supervisor de Producción,PRODUCCION,Mandos Medios,Grijalva Ochoa


3- En el pair programming desde a dirección nos pedían que agruparamos los datos en base a la columna de "categoría", puede que en aquel momento nos dieramos cuenta que había varías categorías que eran igual (en concreto "Acceso Login") pero que en algunos casos tenía unos símbolos raros (&, %, -). 

Es el momento de limpiar esta columna. El objetivo, reemplazar todos estos símbolos raros por "/", para conseguir homogeneizar los valores de esta columna. Para ellos deberemos crear una función.

In [15]:
df.sample(2)

,id_ticket,fecha,id_empleado,id_agente,categoría,tipo,severidad,prioridad,días_resolución,satisfacción,semestre,año,mes
7650,KHLESR-6343256017,2018-06-05,201,20,Sistema,Problema,2 - Normal,3 - Alta,3,5,1 semestre,2018,June
2031,GHLESR-0643358595,2018-09-15,1509,35,Sistema,Problema,2 - Normal,3 - Alta,6,5,2º semestre,2018,September


In [16]:
categorías = df['categoría'].unique()
categorías

array(['Acceso&%Login', 'Sistema', 'Software', 'Hardware',
       'Acceso%&Login', 'Acceso-&Login', 'Acceso99Login', 'Acceso/Login'],
      dtype=object)

In [17]:
def limpiar_cat(c):
    if c == 'Acceso&%Login' or c == 'Acceso%&Login' or c == 'Acceso-&Login' or c == 'Acceso99Login':
        return 'Acceso/Login'
    else:
        return c

In [18]:
df['categoría'] = df['categoría'].apply(limpiar_cat)

In [19]:
df['categoría'].unique()

array(['Acceso/Login', 'Sistema', 'Software', 'Hardware'], dtype=object)

4- Una vez que hayáis terminado eliminad las columnas originales conservando las nuevas.

In [20]:
df.sample(4)

,id_ticket,fecha,id_empleado,id_agente,categoría,tipo,severidad,prioridad,días_resolución,satisfacción,semestre,año,mes
20572,SHLTST-1844059242,2020-08-16,14,42,Sistema,Solicitud,2 - Normal,3 - Alta,8,4,2º semestre,2020,August
18352,SDLTSR-8343958938,2020-05-07,393,49,Sistema,Solicitud,2 - Normal,0 - Sin Asignar,13,5,1 semestre,2020,May
11098,KMLTSR-2643482860,2019-01-17,356,18,Sistema,Solicitud,2 - Normal,2 - Media,5,2,1 semestre,2019,January
7640,KHLESR-5443144382,2018-02-13,218,23,Sistema,Problema,2 - Normal,3 - Alta,4,5,1 semestre,2018,February


5- Guardad el dataframe en un csv que usaremos más adelante

In [21]:
df.to_csv("../datos/tickets_final.csv")
dfe.to_csv("../datos/empleados_final.csv")